Мета:

Дослідити різні архітектури генеративно-змагальних мереж (GAN) і практикуватися з налаштуванням параметрів для досягнення кращих результатів генерації зображень.

Кроки для виконання завдання:

1. Вибір архітектур GAN:
  * DCGAN (Deep Convolutional GAN): Реалізуйте DCGAN, що використовує згорткові шари для кращого захоплення просторових властивостей зображень.
  * WGAN (Wasserstein GAN): Спробуйте реалізувати WGAN для покращення стабільності навчання та якості генерації.
  * Conditional GAN (cGAN): Реалізуйте Conditional GAN, щоб навчитися генерувати зображення з додатковими умовами (наприклад, категорією).
2. Підготовка набору даних:
  * Як в попереднізавданнях
3. Налаштування архітектури кожного GAN:
  * Генератор: Експериментуйте з кількістю шарів, розмірами фільтрів, активаціями (наприклад, ReLU, LeakyReLU).
  * Дискримінатор: Вивчіть вплив використання різних функцій активації, шару нормалізації пакетів (Batch Normalization) тощо.
  * Вибір функцій втрат: Для кожної архітектури використовуйте відповідні функції втрат (наприклад, binary cross-entropy для базового GAN, Wasserstein loss для WGAN).
4. Методи налаштування (fine-tuning):
  * Розмір батчу: Спробуйте різні розміри батчу для кращої стабільності навчання.
  * Коефіцієнт навчання: Експериментуйте з різними значеннями коефіцієнта навчання (learning rate) для генератора і дискримінатора.
  * Регуляризація: Додайте Dropout до дискримінатора, щоб запобігти перенавчанню.
  * Зміна оптимізаторів: Використайте оптимізатори Adam, RMSprop або інші для покращення швидкості збіжності.
5. Навчання та порівняння архітектур:
  * Навчіть кожну з архітектур на одному наборі даних для порівняння результатів.
  * Періодично зберігайте згенеровані зображення для візуальної оцінки прогресу.
6. Оцінка та візуалізація результатів:
  * Візуалізуйте згенеровані зображення для кожної з архітектур GAN.
  * Порівняйте результати на основі якості та різноманітності згенерованих зображень.
  * Проведіть аналіз, яка архітектура дає найкращі результати для обраного набору даних.
7. Документування висновків:
  * Оцініть, яка архітектура показала найкращі результати, та опишіть, чому це сталося.
  * Опишіть, які методи налаштування мали найбільший вплив на продуктивність моделей.

Мінімальні вимоги:

* Реалізувати та налаштувати принаймні дві різні архітектури GAN.
* Візуально оцінити та порівняти результати кожної архітектури.

Додаткові рекомендації:

* Експериментуйте з використанням більш складних наборів даних для оцінки ефективності кожної архітектури.

Формат виконання: 

.ipynb блокнот з кодом та візуалізацією, або ж код в .py з прикріпленими зображеннями результатів на гітхабі.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np

In [2]:
# Example usage
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load your dataset here (example with CIFAR-10)
dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2)

DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

Files already downloaded and verified


In [3]:
# Generator Network
class Generator(nn.Module):
    def __init__(self, latent_dim=100, channels=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input is latent_dim x 1 x 1
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            # State size: 512 x 4 x 4
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # State size: 256 x 8 x 8
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # State size: 128 x 16 x 16
            nn.ConvTranspose2d(128, channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output size: channels x 32 x 32
        )

    def forward(self, x):
        return self.main(x)

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input size: channels x 32 x 32
            nn.Conv2d(channels, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: 128 x 16 x 16
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: 256 x 8 x 8
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: 512 x 4 x 4
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1).squeeze(1)

# Training setup
def train_dcgan(dataloader, num_epochs=100, latent_dim=100):    
    # Initialize networks
    generator = Generator(latent_dim).to(DEVICE)
    discriminator = Discriminator().to(DEVICE)
    
    # Setup optimizers
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    criterion = nn.BCELoss()
    
    for epoch in range(num_epochs):
        for i, (real_images, _) in enumerate(dataloader):
            batch_size = real_images.size(0)
            real_label = torch.ones(batch_size).to(DEVICE)
            fake_label = torch.zeros(batch_size).to(DEVICE)
            
            # Train Discriminator
            real_images = real_images.to(DEVICE)
            d_optimizer.zero_grad()
            output = discriminator(real_images)
            d_loss_real = criterion(output, real_label)
            
            noise = torch.randn(batch_size, latent_dim, 1, 1).to(DEVICE)
            fake_images = generator(noise)
            output = discriminator(fake_images.detach())
            d_loss_fake = criterion(output, fake_label)
            
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()
            
            # Train Generator
            g_optimizer.zero_grad()
            output = discriminator(fake_images)
            g_loss = criterion(output, real_label)
            g_loss.backward()
            g_optimizer.step()
            
            if i % 100 == 0:
                print(f'Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'd_loss: {d_loss.item():.4f} g_loss: {g_loss.item():.4f}')
    
    return generator, discriminator


# Train the model
generatorDGAN, discriminatorDGAN = train_dcgan(dataloader,num_epochs=5)


Epoch [0/5] Batch [0/391] d_loss: 1.3821 g_loss: 3.4049
Epoch [0/5] Batch [100/391] d_loss: 0.3620 g_loss: 0.5470
Epoch [0/5] Batch [200/391] d_loss: 0.3008 g_loss: 3.4415
Epoch [0/5] Batch [300/391] d_loss: 0.8480 g_loss: 2.1030
Epoch [1/5] Batch [0/391] d_loss: 1.0325 g_loss: 5.5890
Epoch [1/5] Batch [100/391] d_loss: 0.3986 g_loss: 3.5943
Epoch [1/5] Batch [200/391] d_loss: 0.8033 g_loss: 1.8956
Epoch [1/5] Batch [300/391] d_loss: 0.3237 g_loss: 3.4196
Epoch [2/5] Batch [0/391] d_loss: 0.9655 g_loss: 0.5806
Epoch [2/5] Batch [100/391] d_loss: 0.5430 g_loss: 2.7731
Epoch [2/5] Batch [200/391] d_loss: 0.3582 g_loss: 4.3093
Epoch [2/5] Batch [300/391] d_loss: 0.6665 g_loss: 3.9667
Epoch [3/5] Batch [0/391] d_loss: 0.0643 g_loss: 4.2148
Epoch [3/5] Batch [100/391] d_loss: 0.3651 g_loss: 4.5731
Epoch [3/5] Batch [200/391] d_loss: 0.4907 g_loss: 4.7940
Epoch [3/5] Batch [300/391] d_loss: 1.2277 g_loss: 5.7699
Epoch [4/5] Batch [0/391] d_loss: 0.4882 g_loss: 4.0337
Epoch [4/5] Batch [100/3

In [ ]:

class Generator(nn.Module):
    def __init__(self, latent_dim=100, channels=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(128, channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

class Critic(nn.Module):
    def __init__(self, channels=3):
        super(Critic, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(channels, 128, 4, 2, 1),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(512, 1, 4, 1, 0)
        )

    def forward(self, x):
        return self.main(x).view(-1)

def train_wgan(dataloader, num_epochs=100, latent_dim=100, critic_iterations=5, clip_value=0.01):    
    generator = Generator(latent_dim).to(DEVICE)
    critic = Critic().to(DEVICE)
    
    g_optimizer = optim.RMSprop(generator.parameters(), lr=0.00005)
    c_optimizer = optim.RMSprop(critic.parameters(), lr=0.00005)
    
    for epoch in range(num_epochs):
        for i, (real_images, _) in enumerate(dataloader):
            batch_size = real_images.size(0)
            real_images = real_images.to(DEVICE)
            
            # Train Critic
            for _ in range(critic_iterations):
                noise = torch.randn(batch_size, latent_dim, 1, 1).to(DEVICE)
                fake_images = generator(noise)
                
                c_optimizer.zero_grad()
                
                # Wasserstein loss
                critic_real = critic(real_images).mean()
                critic_fake = critic(fake_images.detach()).mean()
                critic_loss = -critic_real + critic_fake
                
                critic_loss.backward()
                c_optimizer.step()
                
                # Weight clipping
                for p in critic.parameters():
                    p.data.clamp_(-clip_value, clip_value)
            
            # Train Generator
            g_optimizer.zero_grad()
            fake_images = generator(noise)
            generator_loss = -critic(fake_images).mean()
            
            generator_loss.backward()
            g_optimizer.step()
            
            if i % 100 == 0:
                print(f'Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] '
                      f'Critic Loss: {critic_loss.item():.4f} '
                      f'Generator Loss: {generator_loss.item():.4f}')
    
    return generator, critic

# Data loading and training setup

generatorWGAN, criticWGAN = train_wgan(dataloader,num_epochs=5)



Epoch [0/5] Batch [0/391] Critic Loss: -29.4890 Generator Loss: 19.7102


In [ ]:
# Generate sample images
def generate_samples(generator, latent_dim=100, num_samples=16):
    with torch.no_grad():
        noise = torch.randn(num_samples, latent_dim, 1, 1).to(DEVICE)
        fake_images = generator(noise)
        
        import matplotlib.pyplot as plt
        import torchvision.utils as vutils
        
        plt.figure(figsize=(10,10))
        plt.imshow(np.transpose(vutils.make_grid(
            fake_images.cpu(), padding=2, normalize=True), (1,2,0)))
        plt.axis('off')
        plt.show()

generate_samples(generatorDGAN)
generate_samples(generatorWGAN)


Key differences WGAN from DCGAN:

Uses a Critic instead of Discriminator (no sigmoid at the end)
Implements Wasserstein loss instead of BCE
Uses weight clipping for Lipschitz constraint
Employs RMSprop optimizer instead of Adam
Multiple critic updates per generator update
Uses InstanceNorm instead of BatchNorm in the critic